In [1]:
## library imports

## db connection params
from config import get_engine

## IO
import pandas as pd

## linear algebra
import numpy as np

## regex 
import re 

## timimg
import time

from processing import strip, replace_empty_string,map_season,replace_finished_game_vals

In [2]:
## db engine
engine = get_engine("landing")
processing_engine = get_engine('processing')

In [3]:
## getting curling linescores from raspberry pi MySql DB

teams = pd.read_sql("SELECT * FROM Team;",con=engine)

In [4]:
cols = ['Player','Position','TeamClub']

In [5]:
## strip cols

teams[cols] = teams[cols].applymap(strip)

In [6]:
## replace empty strings with null values

teams[cols] = teams[cols].applymap(replace_empty_string)

In [7]:
## drop entries where player name is null

teams = teams.loc[teams.Player.notnull()].reset_index(drop=True)

In [8]:
## add year

pat = r"([S|A|1|2|3|4])(?=:)"
positions = teams['Position'].str.extract(pat=pat)
assert positions.isnull().sum().sum() == 0
teams['Position'] = positions.copy()

In [9]:
## get TeamID

pat = r"^.*teamid=(\d*)"
team_ids = teams['TeamLink'].str.extract(pat=pat,expand=False)
assert team_ids.isnull().sum() == 0

teams['TeamID'] = team_ids.copy()

In [11]:
## sql

index = False
if_exists = 'append'
teams.to_sql("Team",con=processing_engine,index=index,if_exists=if_exists)